# 0.0 - Importando as bibliotecas

In [1]:
import pandas         as pd
import seaborn        as sns
import plotly.express as px
import ipywidgets     as widgets

from ipywidgets            import fixed
from matplotlib            import gridspec
from matplotlib            import pyplot as plt
from geopy.geocoders       import Nominatim
from IPython.core.display  import HTML
from IPython.display       import Image


## 0.1 - Funções

In [2]:
def jupyter_settings():
    
    %matplotlib inline 
    #%pylab inline
    
    plt.style.use('bmh')
    plt.rcParams['figure.figsize']=[20,10]
    plt.rcParams['font.size']=10
    
    display( HTML('<style>.container {width:100% !important; }</style>'))
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option('display.expand_frame_repr',False )
    pd.set_option('display.float_format',lambda x: '%.2f' % x)
    

    sns.set()
    


jupyter_settings()

## 0.2 - Carregando os dados

In [3]:
df = pd.read_csv('../data/kc_house_data.csv')

In [4]:
df.head(5)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.00,3,1.00,1180,5650,1.00,0,0,3,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650
1,6414100192,20141209T000000,538000.00,3,2.25,2570,7242,2.00,0,0,3,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639
2,5631500400,20150225T000000,180000.00,2,1.00,770,10000,1.00,0,0,3,6,770,0,1933,0,98028,47.74,-122.23,2720,8062
3,2487200875,20141209T000000,604000.00,4,3.00,1960,5000,1.00,0,0,5,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000
4,1954400510,20150218T000000,510000.00,3,2.00,1680,8080,1.00,0,0,3,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503


# 1.0 -  Qual a média do preço de compra dos imóveis por "Nível"?

- Nível 0 -> Preço entre  0 e 321.950
- Nível 1 -> Preço entre 321.950 e  450.000
- Nível 2 -> Preço entre 450.000 e 645.000
- Nível 3 -> Acima de 645.000

In [5]:
df['level']='NA'

for i in range(len(df)):
    
    if (df.loc[i,'price'] > 0) & (df.loc[i,'price'] <321950):
        df.loc[i,'level'] = '0'
    
    elif (df.loc[i,'price'] >= 321950) & (df.loc[i,'price'] < 450000):
        df.loc[i,'level'] = '1'
    
    elif (df.loc[i,'price'] >= 450000) & (df.loc[i,'price'] < 650000):
        df.loc[i,'level'] = '2'
     
    else:
        df.loc[i,'level'] = '3'

df['level'] = df['level'].astype(int)

# 2.0 - Qual a média do tamanho da sala de estar dos imóveis por "Size"?

- Size 0 -> Tamanho entre 0 e 1427 sqft
- Size 1 -> Tamanho entre 1427 e 1910 sqft
- Size 2 -> Tamanho entre 1910 e 2550 sqft
- Size 3 -> Tamanho acima de 2550 sqft

In [6]:
df['size']= 'NA'


df['size']= df['sqft_living'].apply(lambda x: 0  if x < 1427 else 
                                              1  if (x >= 1427) & (x < 1910) else 
                                              2  if (x >= 1910) & (x < 2550) else 3)  



df['size'] = df['size'].astype(int)

# 3.0 - Adicione as seguintes informações ao conjunto de dados original:
- PlaceID: identificação da localização.
- OSM Type: Open Street Map Type
- Country: Nome do País
- Country Code: Código do país

In [25]:
df1 = df.copy()

In [26]:
df1.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,level,size,is_waterfront,year,year_week
0,7129300520,2014-10-13,221900.00,3,1.00,1180,5650,1.00,0,0,3,7,1180,0,1955,0,98178,47.51,-122.26,1340,5650,0,0,no,2014,2014-41
1,6414100192,2014-12-09,538000.00,3,2.25,2570,7242,2.00,0,0,3,7,2170,400,1951,1991,98125,47.72,-122.32,1690,7639,2,3,no,2014,2014-49
2,5631500400,2015-02-25,180000.00,2,1.00,770,10000,1.00,0,0,3,6,770,0,1933,0,98028,47.74,-122.23,2720,8062,0,0,no,2015,2015-08
3,2487200875,2014-12-09,604000.00,4,3.00,1960,5000,1.00,0,0,5,7,1050,910,1965,0,98136,47.52,-122.39,1360,5000,2,2,no,2014,2014-49
4,1954400510,2015-02-18,510000.00,3,2.00,1680,8080,1.00,0,0,3,8,1680,0,1987,0,98074,47.62,-122.05,1800,7503,2,1,no,2015,2015-07


In [ ]:
# Create empty rows
df1['country'] = 'NA'
df1['state'] = 'NA'
df1['city'] = 'NA'
df1['neighbourhood'] = 'NA'
df1['road'] = 'NA'
df1['house_number'] = 'NA'

# Initialize Nominatim API
geolocator = Nominatim(user_agent='geoapiExercises')

for i in range(len(df1)):
    
    print('Loop:{} / {}'.format(i,len(df1)))
    
    query = str(df1.loc[i,'lat']) + ','+ str(df1.loc[i,'long'])
    
    #API request
    response = geolocator.reverse(query)
    
    # Populate data
    if 'country' in response.raw['address']:
        df1.loc[i,'country'] = response.raw['address']['country']
           
    if 'state' in response.raw['address']:
        df1.loc[i,'state'] = response.raw['address']['state']
    
    if 'city' in response.raw['address']:
        df1.loc[i,'city'] = response.raw['address']['city']
    
    if 'neighbourhood' in response.raw['address']:
        df1.loc[i,'neighbourhood'] = response.raw['address']['neighbourhood']
    
    if 'road' in response.raw['address']:
        df1.loc[i,'road'] = response.raw['address']['road']
    
    if 'house_number' in response.raw['address']:
        df1.loc[i,'house_number'] = response.raw['address']['house_number']

Loop:0 / 21613
Loop:1 / 21613
Loop:2 / 21613
Loop:3 / 21613
Loop:4 / 21613
Loop:5 / 21613
Loop:6 / 21613
Loop:7 / 21613
Loop:8 / 21613
Loop:9 / 21613
Loop:10 / 21613
Loop:11 / 21613
Loop:12 / 21613
Loop:13 / 21613
Loop:14 / 21613
Loop:15 / 21613
Loop:16 / 21613
Loop:17 / 21613
Loop:18 / 21613
Loop:19 / 21613
Loop:20 / 21613
Loop:21 / 21613
Loop:22 / 21613
Loop:23 / 21613
Loop:24 / 21613
Loop:25 / 21613
Loop:26 / 21613
Loop:27 / 21613
Loop:28 / 21613
Loop:29 / 21613
Loop:30 / 21613
Loop:31 / 21613
Loop:32 / 21613
Loop:33 / 21613
Loop:34 / 21613
Loop:35 / 21613
Loop:36 / 21613
Loop:37 / 21613
Loop:38 / 21613
Loop:39 / 21613
Loop:40 / 21613
Loop:41 / 21613
Loop:42 / 21613
Loop:43 / 21613
Loop:44 / 21613
Loop:45 / 21613
Loop:46 / 21613
Loop:47 / 21613
Loop:48 / 21613
Loop:49 / 21613
Loop:50 / 21613
Loop:51 / 21613
Loop:52 / 21613
Loop:53 / 21613
Loop:54 / 21613
Loop:55 / 21613
Loop:56 / 21613
Loop:57 / 21613
Loop:58 / 21613
Loop:59 / 21613
Loop:60 / 21613
Loop:61 / 21613
Loop:62 / 21613
Lo

# 4.0 - Adicione os seguintes filtros no mapa:
- Tamanho mínimo da área da sala estar.
- Número mínimo de banheiros.
- Valor máximo do Preço.
- Tamanho máximo da área do porão.
- Filtro das condições do imóvel.
- Filtro por ano de construção.


In [7]:
df['is_waterfront'] = df['waterfront'].apply( lambda x: 'yes' if x == 1 else 'no' )

style = {'description_width': 'initial'}

#Botões interativos
price_limit = widgets.IntSlider( 
    value = 540000,
    min = 75000, 
    max = 77000000, 
    step = 1,
    description='Maximun Price',
    disable=False,
    style = style
)

waterfront_bar = widgets.Dropdown( 
    options= df['is_waterfront'].unique().tolist(), 
    value='yes', 
    description='Water View', 
    disable=False)


def update_map( df, waterfront, limit ):
    
    #Definindo as colunas para o mapa
    houses = df[(df['price'] <= limit) & (df['is_waterfront'] == waterfront)][['id','price','lat','long','level']]

    #Criando mapa
    maps_price = px.scatter_mapbox(data_frame= houses,
                lat='lat',
                lon='long',
                hover_name='id',
                color='level',                  
                size='price',                  
                color_discrete_sequence=['darkblue'],
                color_continuous_scale=px.colors.sequential.Viridis,
                size_max=30,                  
                zoom=8,
                mapbox_style='open-street-map',
                height=1000) 

    #Definindo as margens do mapa
    maps_price.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
    maps_price.show()

widgets.interactive( update_map, df=fixed(df), waterfront=waterfront_bar, limit=price_limit)

interactive(children=(Dropdown(description='Water View', index=1, options=('no', 'yes'), value='yes'), IntSlid…

# 5.0 - Adicione os seguintes filtros no Dashboard:
- Filtro por data disponível para compra.
- Filtro por ano de renovação.
- Filtro se possui vista para a água ou não

In [19]:
#Formato de Datas
df['year'] = pd.to_datetime(df['date']).dt.strftime('%Y')
df['date'] = pd.to_datetime(df['date']).dt.strftime('%Y-%m-%d')
df['year_week'] = pd.to_datetime(df['date']).dt.strftime('%Y-%U')

# Widgets to control data
date_limit = widgets.SelectionSlider(
    options=df['date'].sort_values().unique().tolist(),
    value = '2014-12-01',
    description = 'Disponivel',
    continuous_update=False,
    orientation='horizontal',
    readout=True )

def update_map( data, limit ):
    # Filter data
    df = data[data['date'] >= limit ].copy()
    
    fig = plt.figure( figsize=(21,12) )
    specs = gridspec.GridSpec( ncols=2, nrows=2, figure=fig )
    
    ax1 = fig.add_subplot( specs[0, :] ) # First rows
    ax2 = fig.add_subplot( specs[1, 0] ) # Second Row First Column
    ax3 = fig.add_subplot( specs[1, 1] ) # Second Row Second Column  rows
    
    by_year = df[['id', 'year']].groupby( 'year').sum().reset_index()
    ax1.bar( by_year['year'], by_year['id'] )
    
    by_day = df[['id', 'date']].groupby( 'date').mean().reset_index()
    ax2.plot( by_day['date'], by_day['id'] )
    ax2.set_title( 'title: Avg Price by Day' )
    
    by_week_of_year = df2[['id', 'year_week']].groupby( 'year_week').mean().reset_index()
    ax3.bar( by_week_of_year['year_week'], by_week_of_year['id'] )
    ax3.set_title( 'title: Avg Price by Week Of Year' )
    plt.xticks( rotation=60 );
    
widgets.interactive( update_map, data=fixed(data), limit=date_limit)

# 6.0 - Anotações da aula (While) 

In [23]:
import requests as r

i = 1
df_jobs = pd.DataFrame()

while True:
    
    print('page: {}'.format(i))

    url = 'https://jobs.github.com/positions.json?description=+Software+Engineer&location=&page={}'.format(i)
    response = r.request('GET',url)
    
    if response.json() != []:
        data = response.json()[0]
        df2 = pd.DataFrame(data,index=[0])
        
        print(df1[['id','type']])
        
        df_jobs = pd.concat([df_jobs,df1], axis=0)
        
        i = i + 1
        
    else:
        break

page: 1
                                     id       type
0  80a23680-0958-4daa-8e99-dfde25e6e839  Full Time
page: 2
                                     id       type
0  80a23680-0958-4daa-8e99-dfde25e6e839  Full Time
page: 3
                                     id       type
0  80a23680-0958-4daa-8e99-dfde25e6e839  Full Time
page: 4


In [24]:
df_jobs.head()

,id,type,url,created_at,company,company_url,location,title,description,how_to_apply,company_logo,country,state,city,neighbourhood,road,house_number
0,80a23680-0958-4daa-8e99-dfde25e6e839,Full Time,https://jobs.github.com/positions/80a23680-095...,Tue Jan 12 16:04:05 UTC 2021,Scopic Software,None,Remote,Remote DevOps Engineer,<p><strong>Scopic</strong> is seeking a <stron...,"<p>Please apply through a link - <a href=""http...",https://jobs.github.com/rails/active_storage/b...,NA,NA,NA,NA,NA,NA
0,80a23680-0958-4daa-8e99-dfde25e6e839,Full Time,https://jobs.github.com/positions/80a23680-095...,Tue Jan 12 16:04:05 UTC 2021,Scopic Software,None,Remote,Remote DevOps Engineer,<p><strong>Scopic</strong> is seeking a <stron...,"<p>Please apply through a link - <a href=""http...",https://jobs.github.com/rails/active_storage/b...,NA,NA,NA,NA,NA,NA
0,80a23680-0958-4daa-8e99-dfde25e6e839,Full Time,https://jobs.github.com/positions/80a23680-095...,Tue Jan 12 16:04:05 UTC 2021,Scopic Software,None,Remote,Remote DevOps Engineer,<p><strong>Scopic</strong> is seeking a <stron...,"<p>Please apply through a link - <a href=""http...",https://jobs.github.com/rails/active_storage/b...,NA,NA,NA,NA,NA,NA
